In [ ]:
%pip install pandas_ta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 8.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 10.9 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [76]:
# Install missing package in the notebook environment


import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [77]:
df_BTC = pd.read_csv('data_Jack_bianance/1hr/BTCUSDT_1h.csv')
df_DOGE = pd.read_csv('data_Jack_bianance/1hr/DOGEUSDT_1h.csv')

# Trend Detection

### Calculate SMA

In [78]:
def calculate_SMA20(df, column_name='close', period = 20):

    return df[column_name].rolling(window=period).mean()

def calculate_SMA50(df, column_name='close', period = 50):
    
    return df[column_name].rolling(window=period).mean()

### Calculate EMA

In [79]:
def calculate_EMA20(df, column_name: str = 'close', period: int = 20):

    return df[column_name].ewm(span=period, adjust=False).mean()

def calculate_EMA50(df, column_name: str = 'close', period: int = 50):

    return df[column_name].ewm(span=period, adjust=False).mean()

In [80]:
df_BTC['SMA_20'] = calculate_SMA20(df_BTC)
df_BTC['SMA_50'] = calculate_SMA50(df_BTC)

df_BTC['EMA_20'] = calculate_EMA20(df_BTC)
df_BTC['EMA_50'] = calculate_EMA50(df_BTC)

print(df_BTC)

                       timestamp       open       high        low      close  \
0      2020-01-01 00:00:00+00:00    7195.24    7196.25    7175.46    7177.02   
1      2020-01-01 01:00:00+00:00    7176.47    7230.00    7175.71    7216.27   
2      2020-01-01 02:00:00+00:00    7215.52    7244.87    7211.41    7242.85   
3      2020-01-01 03:00:00+00:00    7242.66    7245.00    7220.00    7225.01   
4      2020-01-01 04:00:00+00:00    7225.00    7230.00    7215.03    7217.27   
...                          ...        ...        ...        ...        ...   
51219  2025-11-05 11:00:00+00:00  101401.97  102110.48  101381.68  102070.61   
51220  2025-11-05 12:00:00+00:00  102070.62  102800.71  101916.77  102673.35   
51221  2025-11-05 13:00:00+00:00  102673.34  103253.39  102291.82  102985.91   
51222  2025-11-05 14:00:00+00:00  102985.90  103470.00  102169.06  103202.30   
51223  2025-11-05 15:00:00+00:00  103202.30  103441.10  102829.51  103046.68   

            volume       SMA_20       S

# Find Pivot Points

In [81]:
def find_pivots(df: pd.DataFrame, window: int = 2, min_gap: int = 2) -> pd.Series:
    """
    Detect pivot (fractal) points in OHLC data, then thin so that in any
    `min_gap` consecutive bars there is at most one pivot (of any type).

    Returns:
        pd.Series: 0 = no pivot, 1 = pivot high, 2 = pivot low, 3 = both.
    """
    pivots = [0] * len(df)

    for candle in range(len(df)):
        # skip edges
        if candle - window < 0 or candle + window >= len(df):
            continue

        pivotHigh = True
        pivotLow = True

        # check neighborhood [candle-window, candle+window]
        for i in range(candle - window, candle + window + 1):
            if df.iloc[candle].low > df.iloc[i].low:
                pivotLow = False
            if df.iloc[candle].high < df.iloc[i].high:
                pivotHigh = False

        if pivotHigh and pivotLow:
            pivots[candle] = 3 # both pivot high and low (rare case) 
        elif pivotHigh:
            pivots[candle] = 1  # pivot high
        elif pivotLow:
            pivots[candle] = 2  # pivot low

    # === Spacing enforcement: at most one pivot in any `min_gap` bars ===
    # Treat any non-zero (1,2,3) as a pivot; keep the first, drop others within the next (min_gap-1) bars.
    piv = np.array(pivots, dtype=int)
    last_kept = -10**9  # sufficiently negative

    # indices of any pivot (1,2,3), in chronological order
    pivot_idx = np.where(piv != 0)[0]

    for idx in pivot_idx:
        if idx - last_kept >= min_gap:
            last_kept = idx      # keep this pivot
        else:
            piv[idx] = 0         # too close to previous pivot → drop

    return pd.Series(piv, index=df.index)



In [83]:
df_DOGE['isPivot'] = find_pivots(df_DOGE)

# Finding Support/Resistance

In [ ]:
def _cluster_pivots_by_window(df: pd.DataFrame,
                              idxs: np.ndarray,
                              prices: pd.Series,
                              window_bars: int = 20,
                              min_points: int = 2) -> pd.DataFrame:
    """
    Group pivot indices into clusters where max_index - first_index <= window_bars,
    then take the mean price per cluster. Only keep clusters with >= min_points.
    Returns a DataFrame with columns:
      ['level','n_points','start_idx','end_idx','start_time','end_time']
    """
    if len(idxs) == 0:
        return pd.DataFrame(columns=['level','n_points','start_idx','end_idx','start_time','end_time'])

    clusters = []
    current = [(idxs[0], float(prices.loc[idxs[0]]))]
    cluster_start = idxs[0]

    for idx in idxs[1:]:
        if idx - cluster_start <= window_bars:
            current.append((idx, float(prices.loc[idx])))
        else:
            # finalize previous cluster
            if len(current) >= min_points:
                lvl = np.mean([p for _, p in current])
                start_i, end_i = current[0][0], current[-1][0]
                clusters.append({
                    'level': lvl,
                    'n_points': len(current),
                    'start_idx': start_i,
                    'end_idx': end_i,
                    'start_time': df.loc[start_i, 'timestamp'] if 'timestamp' in df.columns else start_i,
                    'end_time': df.loc[end_i, 'timestamp'] if 'timestamp' in df.columns else end_i,
                })
            # start a new cluster
            current = [(idx, float(prices.loc[idx]))]
            cluster_start = idx

    # finalize last cluster
    if len(current) >= min_points:
        lvl = np.mean([p for _, p in current])
        start_i, end_i = current[0][0], current[-1][0]
        clusters.append({
            'level': lvl,
            'n_points': len(current),
            'start_idx': start_i,
            'end_idx': end_i,
            'start_time': df.loc[start_i, 'timestamp'] if 'timestamp' in df.columns else start_i,
            'end_time': df.loc[end_i, 'timestamp'] if 'timestamp' in df.columns else end_i,
        })

    return pd.DataFrame(clusters)

In [69]:
def compute_sr_levels(df: pd.DataFrame,
                      window_bars: int = 20,
                      min_points: int = 2,
                      include_both_as_both: bool = True):
    """
    Build resistance/support by clustering nearby pivots and averaging their prices.
    - window_bars: 'closeness' in bars (time). Use 20 per your spec.
    - min_points: require at least this many pivots in the cluster (2+).
    - include_both_as_both: treat '3' pivots as both high and low.
    Returns: (resist_df, support_df)
    """
    # pick which pivot codes count
    res_codes = [1, 3] if include_both_as_both else [1]
    sup_codes = [2, 3] if include_both_as_both else [2]

    # indices of pivots
    res_idxs = df.index[df['isPivot'].isin(res_codes)].to_numpy()
    sup_idxs = df.index[df['isPivot'].isin(sup_codes)].to_numpy()

    # prices to average
    res_prices = df['high']
    sup_prices = df['low']

    resist_df = _cluster_pivots_by_window(df, res_idxs, res_prices, window_bars, min_points)
    support_df = _cluster_pivots_by_window(df, sup_idxs, sup_prices, window_bars, min_points)

    resist_df['kind'] = 'resistance'
    support_df['kind'] = 'support'
    return resist_df, support_df

In [70]:
resist_df, support_df = compute_sr_levels(df_BTC, window_bars=20, min_points=2)
print(resist_df.head())
print(support_df.head())

         level  n_points  start_idx  end_idx                 start_time  \
0  7243.500000         4          3       15  2020-01-01 03:00:00+00:00   
1  7222.355000         2         34       52  2020-01-02 10:00:00+00:00   
2  7375.240000         5         58       77  2020-01-03 10:00:00+00:00   
3  7408.985000         4         82      100  2020-01-04 10:00:00+00:00   
4  7508.153333         3        108      123  2020-01-05 12:00:00+00:00   

                    end_time        kind  
0  2020-01-01 15:00:00+00:00  resistance  
1  2020-01-03 04:00:00+00:00  resistance  
2  2020-01-04 05:00:00+00:00  resistance  
3  2020-01-05 04:00:00+00:00  resistance  
4  2020-01-06 03:00:00+00:00  resistance  
         level  n_points  start_idx  end_idx                 start_time  \
0  7157.283333         3          8       28  2020-01-01 08:00:00+00:00   
1  6989.896000         5         31       50  2020-01-02 07:00:00+00:00   
2  7274.406667         3         70       90  2020-01-03 22:00:00+

# Different State

In [84]:
import plotly.graph_objects as go
import numpy as np

def plot_liquidity_fvg_window(out_df, setups, supports, row_start=None, row_end=None,
                              title="BTC — Liquidity Gap / FVG Strategy"):
    df_full = out_df.sort_values('timestamp')
    if row_start is None: row_start = 0
    if row_end   is None: row_end   = len(df_full)
    dfp = df_full.iloc[row_start:row_end]   # DO NOT reset_index

    # Pivot markers on the visible slice
    piv_high = np.where(dfp['isPivot'].isin([1,3]), dfp['high'] * 1.001, np.nan)
    piv_low  = np.where(dfp['isPivot'].isin([2,3]), dfp['low']  * 0.999, np.nan)

    # Visible time bounds
    t_min = dfp['timestamp'].min()
    t_max = dfp['timestamp'].max()

    # Helpers
    def _pos_to_time(pos):
        if pos is None: return None
        pos = int(pos)
        if 0 <= pos < len(df_full):
            return df_full.iloc[pos]['timestamp']
        return None

    def _clip_segment(a, b, lo, hi):
        """Return clipped [x0,x1] within [lo,hi], or (None,None) if no overlap."""
        x0, x1 = min(a, b), max(a, b)
        x0c, x1c = max(x0, lo), min(x1, hi)
        return (x0c, x1c) if x0c <= x1c else (None, None)

    fig = go.Figure()

    # Candlesticks
    fig.add_trace(go.Candlestick(
        x=dfp['timestamp'],
        open=dfp['open'], high=dfp['high'],
        low=dfp['low'], close=dfp['close'],
        name='Candles'
    ))

    # Pivot highs / lows
    fig.add_trace(go.Scatter(
        x=dfp['timestamp'], y=piv_high,
        mode='markers', name='Pivot High',
        marker=dict(symbol='triangle-up', color='red', size=8)
    ))
    fig.add_trace(go.Scatter(
        x=dfp['timestamp'], y=piv_low,
        mode='markers', name='Pivot Low',
        marker=dict(symbol='triangle-down', color='lime', size=8)
    ))

    # Support lines ONLY between the two pivots (clipped to visible window)
    if supports:
        for s in supports:
            # prefer stored times; fall back to positions if needed
            t1 = s.get('first_time')
            t2 = s.get('second_time')
            if t1 is None or t2 is None:
                t1 = _pos_to_time(s.get('first_idx'))
                t2 = _pos_to_time(s.get('second_idx'))
            if t1 is None or t2 is None:
                continue

            x0, x1 = _clip_segment(t1, t2, t_min, t_max)
            if x0 is None:  # no overlap with window
                continue

            lvl = float(s['level'])
            fig.add_trace(go.Scatter(
                x=[x0, x1],
                y=[lvl, lvl],
                mode='lines',
                line=dict(color='green', dash='dot', width=1.8),
                name='Support',
                hovertemplate=f"Support {lvl:.2f}<extra></extra>",
                showlegend=False  # avoid legend spam for many supports
            ))

    # BUY points (0.618 Fib at PH2 time)
    if setups:
        buy_times, buy_prices = [], []
        for st in setups:
            ph2_time = st.get('ph2_time') or _pos_to_time(st.get('ph2_pos'))
            buy_price = st.get('buy_price_0618')
            if ph2_time is None or buy_price is None:
                continue
            if t_min <= ph2_time <= t_max:
                buy_times.append(ph2_time)
                buy_prices.append(buy_price)

                # Optional: PL1↔PH2 fib box for context
                pl1_time = st.get('pl1_time') or _pos_to_time(st.get('pl1_pos'))
                pl1_price = st.get('pl1_price'); ph2_price = st.get('ph2_price')
                if pl1_time and pl1_price and ph2_price:
                    x0, x1 = _clip_segment(pl1_time, ph2_time, t_min, t_max)
                    if x0 is not None:
                        fig.add_shape(
                            type="rect", xref="x", yref="y",
                            x0=x0, x1=x1,
                            y0=min(pl1_price, ph2_price), y1=max(pl1_price, ph2_price),
                            line=dict(color="gold", width=1, dash="dot"),
                            fillcolor="rgba(255,215,0,0.05)"
                        )

        if buy_times:
            fig.add_trace(go.Scatter(
                x=buy_times, y=buy_prices,
                mode='markers+text',
                name='Buy (0.618 Fib)',
                text=["BUY"] * len(buy_times),
                textposition="bottom center",
                marker=dict(symbol='star', size=12, color='gold'),
                hovertemplate="BUY @ 0.618 Fib<br>%{x|%Y-%m-%d %H:%M}<br>Price: %{y:.2f}<extra></extra>"
            ))

    fig.update_layout(
        title=title,
        xaxis_title="Time", yaxis_title="Price",
        xaxis_rangeslider_visible=False,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='left', x=0),
        plot_bgcolor="white"
    )
    return fig

In [85]:
# Ensure your df (e.g., df_BTC) already has OHLC + timestamp
df_DOGE = prep_df(df_DOGE)
df_DOGE = compute_pivots_if_missing(df_BTC, window=10, min_gap=4)

out_df, levels, setups = run_liquidity_gap_fvg(
    df_DOGE,
    tol_pct_level=0.005,   # ±0.5% to define support/resistance via two close pivots
    n_window=20,           # scan window after support
    far_pct_break=0.02,    # PL1 must be >2% below support
    pivot_col='isPivot'
)

print(f"Supports found: {len(levels['supports'])}, Resistances found: {len(levels['resistances'])}")
print(f"Tradable setups found: {len(setups)}")
if setups:
    print(setups[-1])
    # Buy price (0.618)
    print("Buy @ 0.618:", setups[-1]['buy_price_0618'])

/var/folders/p8/by67wr7139zdvwbp6501h9lh0000gn/T/ipykernel_64378/1004059385.py:10: DeprecationWarning:

is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.

/var/folders/p8/by67wr7139zdvwbp6501h9lh0000gn/T/ipykernel_64378/1004059385.py:10: DeprecationWarning:

is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.



Supports found: 2556, Resistances found: 2833
Tradable setups found: 31
{'support_level': 68355.305, 'start_pos': 42432, 'end_pos': 42448, 'pl1_pos': 42445, 'pl1_price': 66835.0, 'ph1_pos': 42443, 'ph1_price': 68100.0, 'ph2_pos': 42448, 'ph2_price': 68180.0, 'buy_price_0618': 67348.79, 'tradable_setup': 1, 'support_first_pos': 42424, 'support_second_pos': 42432, 'support_first_price': 68240.61, 'support_second_price': 68470.0}
Buy @ 0.618: 67348.79


# Plotting to Check

In [86]:
import plotly.graph_objects as go
import numpy as np

def plot_liquidity_fvg_window(out_df, setups, supports, row_start=None, row_end=None,
                              title="BTC — Liquidity Gap / FVG Strategy"):
    df_full = out_df.sort_values('timestamp')
    if row_start is None: row_start = 0
    if row_end   is None: row_end   = len(df_full)
    dfp = df_full.iloc[row_start:row_end]   # DO NOT reset_index

    # Pivot markers on the visible slice
    piv_high = np.where(dfp['isPivot'].isin([1,3]), dfp['high'] * 1.001, np.nan)
    piv_low  = np.where(dfp['isPivot'].isin([2,3]), dfp['low']  * 0.999, np.nan)

    # Visible time bounds
    t_min = dfp['timestamp'].min()
    t_max = dfp['timestamp'].max()

    # Helpers
    def _pos_to_time(pos):
        if pos is None: return None
        pos = int(pos)
        if 0 <= pos < len(df_full):
            return df_full.iloc[pos]['timestamp']
        return None

    def _clip_segment(a, b, lo, hi):
        """Return clipped [x0,x1] within [lo,hi], or (None,None) if no overlap."""
        x0, x1 = min(a, b), max(a, b)
        x0c, x1c = max(x0, lo), min(x1, hi)
        return (x0c, x1c) if x0c <= x1c else (None, None)

    fig = go.Figure()

    # Candlesticks
    fig.add_trace(go.Candlestick(
        x=dfp['timestamp'],
        open=dfp['open'], high=dfp['high'],
        low=dfp['low'], close=dfp['close'],
        name='Candles'
    ))

    # Pivot highs / lows
    fig.add_trace(go.Scatter(
        x=dfp['timestamp'], y=piv_high,
        mode='markers', name='Pivot High',
        marker=dict(symbol='triangle-up', color='red', size=8)
    ))
    fig.add_trace(go.Scatter(
        x=dfp['timestamp'], y=piv_low,
        mode='markers', name='Pivot Low',
        marker=dict(symbol='triangle-down', color='lime', size=8)
    ))

    # Support lines ONLY between the two pivots (clipped to visible window)
    if supports:
        for s in supports:
            # prefer stored times; fall back to positions if needed
            t1 = s.get('first_time')
            t2 = s.get('second_time')
            if t1 is None or t2 is None:
                t1 = _pos_to_time(s.get('first_idx'))
                t2 = _pos_to_time(s.get('second_idx'))
            if t1 is None or t2 is None:
                continue

            x0, x1 = _clip_segment(t1, t2, t_min, t_max)
            if x0 is None:  # no overlap with window
                continue

            lvl = float(s['level'])
            fig.add_trace(go.Scatter(
                x=[x0, x1],
                y=[lvl, lvl],
                mode='lines',
                line=dict(color='green', dash='dot', width=1.8),
                name='Support',
                hovertemplate=f"Support {lvl:.2f}<extra></extra>",
                showlegend=False  # avoid legend spam for many supports
            ))

    # BUY points (0.618 Fib at PH2 time)
    if setups:
        buy_times, buy_prices = [], []
        for st in setups:
            ph2_time = st.get('ph2_time') or _pos_to_time(st.get('ph2_pos'))
            buy_price = st.get('buy_price_0618')
            if ph2_time is None or buy_price is None:
                continue
            if t_min <= ph2_time <= t_max:
                buy_times.append(ph2_time)
                buy_prices.append(buy_price)

                # Optional: PL1↔PH2 fib box for context
                pl1_time = st.get('pl1_time') or _pos_to_time(st.get('pl1_pos'))
                pl1_price = st.get('pl1_price'); ph2_price = st.get('ph2_price')
                if pl1_time and pl1_price and ph2_price:
                    x0, x1 = _clip_segment(pl1_time, ph2_time, t_min, t_max)
                    if x0 is not None:
                        fig.add_shape(
                            type="rect", xref="x", yref="y",
                            x0=x0, x1=x1,
                            y0=min(pl1_price, ph2_price), y1=max(pl1_price, ph2_price),
                            line=dict(color="gold", width=1, dash="dot"),
                            fillcolor="rgba(255,215,0,0.05)"
                        )

        if buy_times:
            fig.add_trace(go.Scatter(
                x=buy_times, y=buy_prices,
                mode='markers+text',
                name='Buy (0.618 Fib)',
                text=["BUY"] * len(buy_times),
                textposition="bottom center",
                marker=dict(symbol='star', size=12, color='gold'),
                hovertemplate="BUY @ 0.618 Fib<br>%{x|%Y-%m-%d %H:%M}<br>Price: %{y:.2f}<extra></extra>"
            ))

    fig.update_layout(
        title=title,
        xaxis_title="Time", yaxis_title="Price",
        xaxis_rangeslider_visible=False,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='left', x=0),
        plot_bgcolor="white"
    )
    return fig

In [87]:
# Use FULL out_df, not the reset-index slice
fig = plot_liquidity_fvg_window(
    out_df,                       # full df from run_liquidity_gap_fvg
    setups=setups,                # setups list from run_liquidity_gap_fvg
    supports=levels['supports'],  # supports list from run_liquidity_gap_fvg
    row_start=6000, row_end=8000  # pick your window safely
)
fig.show()

# BackTesting

In [88]:
import numpy as np
import pandas as pd

def _find_entry_after_ph2(df: pd.DataFrame, ph2_pos: int, entry_price: float):
    """
    Scan forward from ph2_pos for the first bar whose range includes entry_price.
    Returns (entry_pos, entry_time) or (None, None) if never touched.
    """
    n = len(df)
    for pos in range(ph2_pos + 1, n):
        low = float(df.iloc[pos]['low'])
        high = float(df.iloc[pos]['high'])
        if low <= entry_price <= high:
            t = df.iloc[pos]['timestamp'] if 'timestamp' in df.columns else pos
            return pos, t
    return None, None

def _simulate_trade_long(df: pd.DataFrame,
                         entry_pos: int,
                         entry_price: float,
                         pl1_price: float,
                         support_level: float,
                         ph2_price: float):
    """
    Simulate the management rules on bars AFTER entry_pos.
    Returns a dict with realized return, flags (tp1/2/3 hit), and exit info.
    """
    # Levels
    tp1 = ph2_price
    tp2 = (ph2_price - pl1_price) * 1.618 + pl1_price
    tp3 = (ph2_price - pl1_price) * 2.618 + pl1_price

    # Initial stop: average of PL1 and support
    stop = (pl1_price + support_level) / 2.0

    # Position sizing (total = 1.0 “all capital”)
    remaining = 1.0
    realized = 0.0

    # Flags
    hit_tp1 = False
    hit_tp2 = False
    hit_tp3 = False

    # After TP1, stop moves to ENTRY; after TP2, stop moves to PH2
    n = len(df)
    for pos in range(entry_pos + 1, n):
        low = float(df.iloc[pos]['low'])
        high = float(df.iloc[pos]['high'])
        time = df.iloc[pos]['timestamp'] if 'timestamp' in df.columns else pos

        # SEQUENCE: for a long, evaluate downside (stop) before upside (TP) per bar
        # (conservative fill assumption)
        if not hit_tp1:
            # Stop check
            if low <= stop:
                # exit ALL at stop
                realized += remaining * ((stop / entry_price) - 1.0)
                remaining = 0.0
                return {
                    'exit_pos': pos, 'exit_time': time, 'exit_price': stop,
                    'ret': realized,
                    'hit_tp1': hit_tp1, 'hit_tp2': hit_tp2, 'hit_tp3': hit_tp3
                }
            # TP1 check
            if high >= tp1:
                # sell 50%
                sell_frac = 0.5 * remaining
                realized += sell_frac * ((tp1 / entry_price) - 1.0)
                remaining -= sell_frac
                hit_tp1 = True
                # move stop to entry
                stop = entry_price
                # continue to next bar
                continue

        elif hit_tp1 and not hit_tp2:
            # Stop @ entry
            if low <= stop:
                # exit remaining at entry (breakeven)
                realized += remaining * ((stop / entry_price) - 1.0)
                remaining = 0.0
                return {
                    'exit_pos': pos, 'exit_time': time, 'exit_price': stop,
                    'ret': realized,
                    'hit_tp1': hit_tp1, 'hit_tp2': hit_tp2, 'hit_tp3': hit_tp3
                }
            # TP2
            if high >= tp2:
                sell_frac = 0.5 * remaining  # half of the remainder
                realized += sell_frac * ((tp2 / entry_price) - 1.0)
                remaining -= sell_frac
                hit_tp2 = True
                # move stop to PH2
                stop = ph2_price
                continue

        elif hit_tp2 and not hit_tp3:
            # Stop @ PH2
            if low <= stop:
                # exit remaining at PH2
                realized += remaining * ((stop / entry_price) - 1.0)
                remaining = 0.0
                return {
                    'exit_pos': pos, 'exit_time': time, 'exit_price': stop,
                    'ret': realized,
                    'hit_tp1': hit_tp1, 'hit_tp2': hit_tp2, 'hit_tp3': hit_tp3
                }
            # TP3
            if high >= tp3:
                # exit ALL remaining
                realized += remaining * ((tp3 / entry_price) - 1.0)
                remaining = 0.0
                hit_tp3 = True
                return {
                    'exit_pos': pos, 'exit_time': time, 'exit_price': tp3,
                    'ret': realized,
                    'hit_tp1': hit_tp1, 'hit_tp2': hit_tp2, 'hit_tp3': hit_tp3
                }

    # If we reach here: no exit rule hit before data ended — close at last close
    last_close = float(df.iloc[-1]['close'])
    realized += remaining * ((last_close / entry_price) - 1.0)
    return {
        'exit_pos': len(df) - 1,
        'exit_time': df.iloc[-1]['timestamp'] if 'timestamp' in df.columns else len(df) - 1,
        'exit_price': last_close,
        'ret': realized,
        'hit_tp1': hit_tp1, 'hit_tp2': hit_tp2, 'hit_tp3': hit_tp3
    }

def backtest_liquidity_fvg(out_df: pd.DataFrame, setups: list):
    """
    Run the trade simulation across all setups (in chronological order).
    Returns (trades_df, summary_stats).
    """
    if not setups:
        return pd.DataFrame(), {
            'final_return': 0.0, 'average_return': 0.0, 'success_rate': 0.0,
            'tp1_hits': 0, 'tp2_hits': 0, 'tp3_hits': 0, 'max_drawdown': 0.0,
            'n_trades': 0
        }

    # Ensure chronological order by PH2 (confirmation) time/pos
    df = out_df.sort_values('timestamp').reset_index(drop=True)
    enriched = []
    for s in setups:
        # Positions are positional (from earlier code). Safety clamps:
        ph2_pos = int(s['ph2_pos'])
        pl1_pos = int(s['pl1_pos'])
        if ph2_pos >= len(df) or pl1_pos >= len(df):  # skip out-of-range
            continue

        entry_price = s['buy_price_0618']
        entry_pos, entry_time = _find_entry_after_ph2(df, ph2_pos, entry_price)
        if entry_pos is None:
            enriched.append({
                'status': 'no_entry',
                'entry_time': None, 'entry_pos': None, 'entry_price': entry_price,
                'pl1_price': s['pl1_price'],
                'support_level': s['support_level'],
                'ph2_price': s['ph2_price'],
                'ret': 0.0,
                'hit_tp1': False, 'hit_tp2': False, 'hit_tp3': False
            })
            continue

        # simulate
        result = _simulate_trade_long(
            df=df,
            entry_pos=entry_pos,
            entry_price=entry_price,
            pl1_price=s['pl1_price'],
            support_level=s['support_level'],
            ph2_price=s['ph2_price']
        )

        enriched.append({
            'status': 'filled',
            'entry_time': entry_time, 'entry_pos': entry_pos, 'entry_price': entry_price,
            'pl1_price': s['pl1_price'],
            'support_level': s['support_level'],
            'ph2_price': s['ph2_price'],
            'exit_time': result['exit_time'], 'exit_pos': result['exit_pos'], 'exit_price': result['exit_price'],
            'ret': result['ret'],
            'hit_tp1': result['hit_tp1'], 'hit_tp2': result['hit_tp2'], 'hit_tp3': result['hit_tp3']
        })

    trades_df = pd.DataFrame(enriched)

    # Keep only executed trades for stats (status == 'filled')
    exec_df = trades_df[trades_df['status'] == 'filled'].copy()
    n_trades = len(exec_df)

    if n_trades == 0:
        return trades_df, {
            'final_return': 0.0, 'average_return': 0.0, 'success_rate': 0.0,
            'tp1_hits': 0, 'tp2_hits': 0, 'tp3_hits': 0, 'max_drawdown': 0.0,
            'n_trades': 0
        }

    # Returns are per-trade percentage returns (e.g., 0.12 = +12%)
    avg_ret = float(exec_df['ret'].mean())
    success_rate = float((exec_df['ret'] > 0).mean())

    # Count how many trades hit each TP at any time in the lifecycle
    tp1_hits = int(exec_df['hit_tp1'].sum())
    tp2_hits = int(exec_df['hit_tp2'].sum())
    tp3_hits = int(exec_df['hit_tp3'].sum())

    # Equity curve (compounded, sequential) for max drawdown
    # Assume we reinvest all equity each trade in chronological order of entry_time.
    exec_df = exec_df.sort_values('entry_time')
    equity = [1.0]
    for r in exec_df['ret']:
        equity.append(equity[-1] * (1.0 + r))
    equity = np.array(equity[1:])  # drop initial 1.0 placeholder

    if len(equity) == 0:
        mdd = 0.0
        final_ret = 0.0
    else:
        peaks = np.maximum.accumulate(equity)
        drawdowns = (equity - peaks) / peaks
        mdd = float(drawdowns.min()) if len(drawdowns) else 0.0
        final_ret = float(equity[-1] - 1.0)

    summary = {
        'final_return': final_ret,        # compounded return across trades
        'average_return': avg_ret,        # mean per-trade return
        'success_rate': success_rate,     # fraction of profitable trades
        'tp1_hits': tp1_hits,
        'tp2_hits': tp2_hits,
        'tp3_hits': tp3_hits,
        'max_drawdown': mdd,              # negative value (e.g., -0.23 = -23%)
        'n_trades': n_trades
    }

    return trades_df, summary

In [89]:
trades_df, summary = backtest_liquidity_fvg(out_df, setups)

print("===== Summary =====")
for k, v in summary.items():
    if isinstance(v, float):
        print(f"{k:>16}: {v:.4f}")
    else:
        print(f"{k:>16}: {v}")

print("\n===== Sample trades =====")
print(trades_df.head(10))

===== Summary =====
    final_return: 0.4016
  average_return: 0.0134
    success_rate: 0.4231
        tp1_hits: 10
        tp2_hits: 8
        tp3_hits: 1
    max_drawdown: -0.0315
        n_trades: 26

===== Sample trades =====
     status          entry_time  entry_pos  entry_price  pl1_price  \
0    filled 2020-03-12 10:00:00     1707.0   7054.60854    6924.74   
1    filled 2020-03-15 14:00:00     1783.0   5260.97834    5055.13   
2  no_entry                 NaT        NaN   6043.59790    5866.56   
3  no_entry                 NaT        NaN   6043.59790    5866.56   
4    filled 2020-04-08 13:00:00     2358.0   7208.02600    7077.00   
5  no_entry                 NaT        NaN   6727.65182    6468.27   
6    filled 2020-06-11 15:00:00     3894.0   9536.54428    9372.46   
7    filled 2020-06-26 07:00:00     4246.0   9133.17150    9009.69   
8    filled 2020-12-11 00:00:00     8267.0  18111.52484   17911.12   
9    filled 2020-12-23 21:00:00     8572.0  23302.78000   22810.00   


In [75]:
# After running your strategy:
print(f"Total tradable setups detected: {len(setups)}")

Total tradable setups detected: 31
